<a href="https://colab.research.google.com/github/ismail-benlaredj-idealab/shared/blob/main/LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
!pip install -q seaborn

In [75]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [76]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.18.0


In [77]:
import pandas as pd

url ="/content/banking_synthetic_colab.csv"
raw_dataset = pd.read_csv(url,
                         header=0,  # Use first row as headers
                         na_values='?',
                         comment='\t',
                         sep=',',  # Changed separator to comma
                         skipinitialspace=True,
                         )
raw_dataset = raw_dataset.drop(columns=['RowNumber','CustomerId'])




In [78]:
dataset = raw_dataset.copy()
dataset = pd.get_dummies(dataset, columns=['Surname', "Geography", "Gender","CardType"], prefix='', prefix_sep='')
# dataset.tail()

In [ ]:
dataset.isna().sum()
dataset = dataset.dropna()
# dataset.tail()

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
train_dataset.describe().transpose()

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop(train_features.columns[-1])
test_labels = test_features.pop(test_features.columns[-1])



selected_feature_names = ['CreditScore', 'Age','SatisfactionScore','Balance', 'Tenure','Complain', 'Exited' ]
train_features = train_features[selected_feature_names]
test_features = test_features[selected_feature_names]



In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
normalizer.adapt(np.array(train_features))



In [ ]:
print(normalizer.mean.numpy())

In [ ]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

linear_model.predict(train_features[:10])

linear_model.layers[1].kernel

linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

linear_model.summary()

history = linear_model.fit(
    train_features,
    train_labels,
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 2])
  plt.xlabel('Epoch')
  plt.ylabel('Error [PointEarned]')
  plt.legend()
  plt.grid(True)

plot_loss(history)

In [ ]:
test_results = {}
test_results['linear_model'] = linear_model.evaluate(
   test_features , test_labels, verbose=0)

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [PointEarned]']).T

In [ ]:
test_predictions = linear_model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [PointEarned]')
plt.ylabel('Predictions [PointEarned]')
lims = [-0.02, 0.05]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)
